In [ ]:
import numpy as np
import random

In [ ]:
cards = ["Ace", "King", "Queen", "Jack", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
signs = ["Club", "Diamond", "Heart", "Spade"]
initialW = 100
n = 1
m = 13
it = int(1e4)
lr = .2
exp = .4
decay = 1.
maxBet = 10
retreat = 1

In [ ]:
class Cards: 
    
    def __init__(self, firstPlayer, secondPlayer):
        """Define the deck of cards"""
        self.cards = 4*list(range(n, m+1))
        self.firstPlayer = firstPlayer
        self.secondPlayer = secondPlayer
        self.turn = 0
        
    def deal(self):
        """Distribution of cards by the agent :)"""
        random.shuffle(self.cards)
        self.turn = 1 - self.turn
        self.firstPlayer.hand = [self.cards[0], self.cards[2]]
        self.secondPlayer.hand = [self.cards[1], self.cards[3]]
            
    def winner(self):
        
        if self.turn < 1:
            if self.secondPlayer.bet == self.firstPlayer.bet:
                if sum(self.firstPlayer.hand) > sum(self.secondPlayer.hand):
                    st = str(self.secondPlayer.hand) +"-"+ str(self.firstPlayer.bet)
                    if self.secondPlayer.statDict.get(st) is None:
                        self.secondPlayer.statDict[st] = {}
                    reward = self.firstPlayer.bet
                    self.secondPlayer.worth -= reward
                    if self.secondPlayer.statDict.get(st).get(str(self.secondPlayer.bet)) is None:
                        self.secondPlayer.statDict[st][str(self.secondPlayer.bet)] = 0
                    self.secondPlayer.statDict[st][str(self.secondPlayer.bet)] += lr * (- decay * reward - self.secondPlayer.statDict[st][str(self.secondPlayer.bet)])

                elif sum(self.firstPlayer.hand) < sum(self.secondPlayer.hand):
                    st = str(self.secondPlayer.hand) +"-"+ str(self.firstPlayer.bet) 
                    if self.secondPlayer.statDict.get(st) is None:
                        self.secondPlayer.statDict[st] = {} 
                    reward = self.firstPlayer.bet
                    self.firstPlayer.worth -= reward
                    if self.secondPlayer.statDict.get(st).get(str(self.secondPlayer.bet)) is None:
                        self.secondPlayer.statDict[st][str(self.secondPlayer.bet)] = 0
                    self.secondPlayer.statDict[st][str(self.secondPlayer.bet)] += lr * (decay * reward - self.secondPlayer.statDict[st][str(self.secondPlayer.bet)])
                    
            elif self.secondPlayer.bet == 0:
                st = str(self.secondPlayer.hand) +"-"+ str(self.firstPlayer.bet) 
                if self.secondPlayer.statDict.get(st) is None:
                    self.secondPlayer.statDict[st] = {}
                reward = retreat
                self.secondPlayer.worth -= reward
                if self.secondPlayer.statDict.get(st).get(str(self.secondPlayer.bet)) is None:
                    self.secondPlayer.statDict[st][str(self.secondPlayer.bet)] = 0
                self.secondPlayer.statDict[st][str(self.secondPlayer.bet)] += lr * (- decay * reward - self.secondPlayer.statDict[st][str(self.secondPlayer.bet)])
            
                
        else:
            if self.firstPlayer.bet == self.secondPlayer.bet:
                if sum(self.firstPlayer.hand) > sum(self.secondPlayer.hand):
                    st = str(self.firstPlayer.hand) +"-"+ str(self.secondPlayer.bet) 
                    if self.firstPlayer.statDict.get(st) is None:
                        self.firstPlayer.statDict[st] = {}
                    reward = self.secondPlayer.bet
                    self.secondPlayer.worth -= reward
                    if self.firstPlayer.statDict.get(st).get(str(self.firstPlayer.bet)) is None:
                        self.firstPlayer.statDict[st][str(self.firstPlayer.bet)] = 0
                    self.firstPlayer.statDict[st][str(self.firstPlayer.bet)] += lr * (decay * reward - self.firstPlayer.statDict[st][str(self.firstPlayer.bet)])
                    
                elif sum(self.firstPlayer.hand) < sum(self.secondPlayer.hand):
                    st = str(self.firstPlayer.hand) +"-"+ str(self.secondPlayer.bet)
                    if self.firstPlayer.statDict.get(st) is None:
                        self.firstPlayer.statDict[st] = {}
                    reward = self.secondPlayer.bet
                    self.firstPlayer.worth -= reward
                    if self.firstPlayer.statDict.get(st).get(str(self.firstPlayer.bet)) is None:
                        self.firstPlayer.statDict[st][str(self.firstPlayer.bet)] = 0
                    self.firstPlayer.statDict[st][str(self.firstPlayer.bet)] += lr * (- decay * reward - self.firstPlayer.statDict[st][str(self.firstPlayer.bet)])

            elif self.firstPlayer.bet == 0:
                st = str(self.firstPlayer.hand) +"-"+ str(self.secondPlayer.bet) 
                if self.firstPlayer.statDict.get(st) is None:
                    self.firstPlayer.statDict[st] = {}
                reward = retreat
                self.firstPlayer.worth -= reward
                if self.firstPlayer.statDict.get(st).get(str(self.firstPlayer.bet)) is None:
                    self.firstPlayer.statDict[st][str(self.firstPlayer.bet)] = 0
                self.firstPlayer.statDict[st][str(self.firstPlayer.bet)] += lr * (- decay * reward - self.firstPlayer.statDict[st][str(self.firstPlayer.bet)])

    def render(self):
        pass
        
    def reset(self):
        self.firstPlayer.reset()
        self.secondPlayer.reset()
        self.turn = random.randint(0, 1)
        
    def play(self, rounds = 100):
        
        score = [0, 0]
        for i in range(rounds):
            self.reset()
            self.deal()
            while self.firstPlayer.worth > 0 and self.secondPlayer.worth > 0:
                if self.turn < 1:
                    actions = list(range(1, min(maxBet, self.firstPlayer.worth, self.secondPlayer.worth)+1))
                    firstAction = self.firstPlayer.chooseAction(actions, None)
                    self.firstPlayer.bet = firstAction
                    actions = [0, firstAction]
                    secondAction = self.secondPlayer.chooseAction(actions, str(self.secondPlayer.hand) +"-"+ str(firstAction))
                    self.secondPlayer.bet = secondAction
                    self.winner()
                
                else:
                    actions = list(range(1, min(maxBet, self.firstPlayer.worth, self.secondPlayer.worth)+1))
                    secondAction = self.secondPlayer.chooseAction(actions, None)
                    self.secondPlayer.bet = secondAction
                    actions = [0, secondAction]
                    firstAction = self.firstPlayer.chooseAction(actions, str(self.firstPlayer.hand) +"-"+ str(secondAction))
                    self.firstPlayer.bet = firstAction
                    self.winner()
                    
                    
                self.deal()
                    
            if self.firstPlayer.worth > 0 and self.secondPlayer.worth < 1:
                score[0] += 1
            elif self.secondPlayer.worth > 0 and self.firstPlayer.worth < 1:
                score[1] += 1
                
        print("THE ULTIMATE SCORE FOR THE TRAINING IS: {0}-{1}".format(score[0], score[1]))
        
    def compete(self):
        self.turn = random.randint(0,1)
        self.deal()
        while self.firstPlayer.worth > 0 and self.secondPlayer.worth > 0:
            if self.turn < 1:
                actions = list(range(1, min(maxBet, self.firstPlayer.worth, self.secondPlayer.worth)+1))
                firstAction = self.firstPlayer.chooseAction(actions, None)
                self.firstPlayer.bet = firstAction
                actions = [0, firstAction]
                secondAction = self.secondPlayer.chooseAction(actions, str(self.secondPlayer.hand) +"-"+ str(firstAction))
                self.secondPlayer.bet = secondAction
                self.winner()
                
            else:
                actions = list(range(1, min(maxBet, self.firstPlayer.worth, self.secondPlayer.worth)+1))
                secondAction = self.secondPlayer.chooseAction(actions, str(self.secondPlayer.hand))
                self.secondPlayer.bet = secondAction
                actions = [0, secondAction]
                firstAction = self.firstPlayer.chooseAction(actions, str(self.firstPlayer.hand) +"-"+ str(secondAction))
                self.firstPlayer.bet = firstAction
                self.winner()
                
            
            print("Adverser Cards and play: {0}, {1}".format(str(self.firstPlayer.hand), str(firstAction)))
            print("Current Net Worth: {0}-{1}".format(self.firstPlayer.worth, self.secondPlayer.worth))
            
            self.deal()

In [ ]:
class Player:
    
    def __init__(self, human = False):
        self.worth = initialW
        self.statDict = {}
        self.bet = maxBet
        
    def reset(self):
        self.worth = initialW
        
    def chooseAction(self, actions, state):
        if state is None:
            return random.choice(actions)
        else:
            if random.random() < exp:
                return random.choice(actions)

            action = None
            vmax = - 1e10

            for act in actions:
                if self.statDict.get(state) is None or self.statDict.get(state).get(str(act)) is None: 
                    current = 0
                else:
                    current = self.statDict.get(state).get(str(act))
                if vmax < current:
                    vmax = current
                    action = act
            return action

In [ ]:
class HumanPlayer(Player):
    
    def __init__(self):
        self.worth = initialW
        self.statDict = {}
        self.bet = maxBet
        
    def reset(self):
        self.worth = initialW
        
    def chooseAction(self, actions, state):
        print("-------------------------")
        print("State: ", state)
        action = int(input("Input your action: ")) 
        
        while action not in actions:
            print("Try a number in ", actions)
            action = int(input("Input your action: "))  
            
        return action

In [ ]:
firstPlayer = Player()
secondPlayer = Player()
cards = Cards(firstPlayer, secondPlayer)
cards.play(it)

In [ ]:
firstPlayer.reset()
human = HumanPlayer()
cards = Cards(firstPlayer, human)
cards.compete()